In [1]:
import tensorflow as tf
import numpy as np
import gnn.gnn_utils as gnn_utils
from gnn.GNN import GNN as GraphNetwork



##### GPU & stuff config
import os

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)


data_path = "./data"

############# training set ################

set_name = "cli_15_7_200"

#inp, arcnode, nodegraph, nodein, labels = Library.set_load_subgraph(data_path, "train")
inp, arcnode, nodegraph, nodein, labels, _ = gnn_utils.set_load_general(data_path, "train", set_name=set_name)
############ test set ####################

#inp_test, arcnode_test, nodegraph_test, nodein_test, labels_test = Library.set_load_subgraph(data_path, "test")
inp_test, arcnode_test, nodegraph_test, nodein_test, labels_test, _ = gnn_utils.set_load_general(data_path, "test", set_name=set_name)

############ validation set #############

#inp_val, arcnode_val, nodegraph_val, nodein_val, labels_val = Library.set_load_subgraph(data_path, "valid")
inp_val, arcnode_val, nodegraph_val, nodein_val, labels_val, _ = gnn_utils.set_load_general(data_path, "validation", set_name=set_name)



EPSILON = 0.00000001

@tf.function()
def loss_fcn(target,output):
    target = tf.cast(target,tf.float32)
    output = tf.maximum(output, EPSILON, name="Avoiding_explosions")  # to avoid explosions
    xent = -tf.reduce_sum(target * tf.math.log(output), 1)
    lo = tf.reduce_mean(xent)
    return lo

@tf.function()
def metric(output, target):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(target, 1))
    metric = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return metric


inp = inp[0]

arcnode=arcnode[0]

nodegraph=nodegraph[0]

inp_val = inp_val[0]

arcnode_val = arcnode_val[0]

nodegraph_val=nodegraph_val[0]

# set input and output dim, the maximum number of iterations, the number of epochs and the optimizer
threshold = 0.001
learning_rate = 0.01
state_dim = 5
input_dim = len(inp[0])
output_dim = 2
max_it = 50
num_epoch = 500


# initialize GNN
param = "st_d" + str(state_dim) + "_th" + str(threshold) + "_lr" + str(learning_rate)
print(param)


model = GraphNetwork(input_dim, state_dim, output_dim,                             
                         hidden_state_dim = 15, hidden_output_dim = 10,
                         ArcNode=arcnode,NodeGraph=None,threshold=threshold)

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer,loss_fcn)



for count in range(0, num_epoch):
    loss_value = model.train_step(inp.astype(np.float32),labels)
    
    if count % 30 == 0:
        #this runs the loop without training
        out_val = model.predict(inp_val.astype(np.float32), arcnode_val)
        loss_value_val = loss_fcn(labels_val,out_val)

        print("Epoch ", count)
        print("Training: ", loss_value.numpy())
        print("Validation: ",loss_value_val.numpy())

        count = count + 1

Epoch  0
Training:  0.8355546
Validation:  0.69346356
Epoch  30
Training:  0.25975695
Validation:  0.24608794
Epoch  60
Training:  0.21404502
Validation:  0.22592442


KeyboardInterrupt: 

In [ ]:
def create_model():

    comp_inp = tf.keras.Input(shape=(input_dim), name="input")
    
    layer = GraphNetwork(input_dim, state_dim, output_dim,                             
                         hidden_state_dim = 15, hidden_output_dim = 10,
                         ArcNode=arcnode,NodeGraph=None,threshold=threshold)
    
    output = layer(comp_inp)
    
    model = tf.keras.Model(comp_inp, output)

    return model,layer


tf.keras.backend.clear_session()
model,GNNLayer = create_model()

# initialize GNN
param = "st_d" + str(state_dim) + "_th" + str(threshold) + "_lr" + str(learning_rate)
print(param)


In [5]:
import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
    self.dropout = tf.keras.layers.Dropout(0.5)

  def call(self, inputs, training=False):
    x = self.dense1(inputs)
    if training:
      x = self.dropout(x, training=training)
    return self.dense2(x)

model = MyModel()

In [6]:
model.compile()

In [7]:
model.fit(np.zeros(6),np.ones(6))

ValueError: in user code:

    C:\Users\rohan\Anaconda3\envs\datasci\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\rohan\Anaconda3\envs\datasci\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\rohan\Anaconda3\envs\datasci\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\rohan\Anaconda3\envs\datasci\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\rohan\Anaconda3\envs\datasci\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\rohan\Anaconda3\envs\datasci\l

In [8]:
model.fit([inp.astype(np.float32)],[labels],epochs=10)

ValueError: Data cardinality is ambiguous:
  x sizes: 15074
  y sizes: 3000
Please provide data which shares the same first dimension.

In [12]:
for count in range(0, num_epoch):
    
    with tf.GradientTape() as tape:
        out = model.call(inp.astype(np.float32))

        loss_value = loss_fcn(labels,out)

        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if count % 30 == 0:
            #this runs the loop without training
            out_val = model.predict_node(inp_val.astype(np.float32), arcnode_val)
            loss_value_val = loss_fcn(labels_val,out_val)
            
            print("Epoch ", count)
            print("Training: ", loss_value.numpy())
            print("Validation: ",loss_value_val.numpy())

        count = count + 1

Epoch  0
Training:  0.07951117
Validation:  0.077302374
Epoch  30
Training:  0.0760324
Validation:  0.0737981
Epoch  60
Training:  0.07362027
Validation:  0.071349435
Epoch  90
Training:  0.0718511
Validation:  0.069538504
Epoch  120
Training:  0.070493236
Validation:  0.06813774


KeyboardInterrupt: 

In [ ]:
model(inp.astype(np.float32))

In [ ]:
 GNNLayer.predict_node(inp.astype(np.float32), arcnode, nodegraph)

In [ ]:
arcnode_val

In [ ]:
12/np.sum(labels)